# KMeans

In [1]:
import cluster_data_5d
from cluster_data_5d import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from cluster_plotter import ClusterPlotter
from clustering_utils_5d import ClusterData
import cluster_plotter
import high_dim_analysis

standard_year_ranges = {
    f"{start}-{start + 3}": np.arange(start, start + 4)
    for start in [2002, 2006, 2010, 2014, 2019]
}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)  # Include 2023
running_ranges = cluster_data_5d.generate_running_year_ranges(2002, 2023, 4)

binned_data = cluster_data_5d.bin_data_for_clustering(running_ranges, print_res=False)

k_values = [5, 6, 7]

results_per_year_range = {}

# Output directory for plots
plot_dir = "Images\k_means_running_bins_5d"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:  
    print(f"\nRunning K-Means for Year Range: {year_range}")

    data_array = np.array([cluster_data.inc, cluster_data.raan, cluster_data.ecc]).T

    normalized_data, data_min, data_max = normalize_data(data_array)

    k_values = [3, 4, 5, 6, 7]

    results = []

    for k in k_values:
        result_kmeans, time_kmeans, n_clusters_kmeans, points_per_cluster_kmeans, metrics_kmeans = run_clustering(
            k_means, f"K-means (k={k})", normalized_data, data_min, data_max, k, init='kmeans++'
        )

        unnormalized_data, cluster_centers = unnormalize(
            result_kmeans.data, result_kmeans.cluster_centers, data_min, data_max
        )

        # Round metric results
        #rounded_cluster_std = {key: tuple(round(val, 3) for val in value) for key, value in metrics_kmeans[4].items()}
        #rounded_square_density = {key: round(value, 3) for key, value in metrics_kmeans[5].items()}
        #rounded_hull_density = {key: round(value, 3) for key, value in metrics_kmeans[6].items()}

        results.append({
            "Year Range": year_range,
            "k": k,
            "Runtime (s)": f"{time_kmeans:.3f}",
            "Clusters": n_clusters_kmeans,
            "Points per Cluster": points_per_cluster_kmeans,
            "Davies-Bouldin": f"{metrics_kmeans[0]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[0], (int, float)) else None,
            "Calinski-Harabasz": f"{metrics_kmeans[1]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[1], (int, float)) else None,
            "Dunn Index": f"{metrics_kmeans[2]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[2], (int, float)) else None,
            "Silhouette Score": f"{metrics_kmeans[3]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[3], (int, float)) else None
        })

        labels = result_kmeans.labels
        """high_dim_analysis.plot_co_membership(
            labels,
            output_folder=plot_dir,
            filename=f"co_membership_{year_range}_k{k}.png"
        )"""
        high_dim_analysis.plot_correlation_heatmap(
            cluster_data,
            output_folder=plot_dir,
            filename=f"corr_{year_range}_k{k}.png"
        )
        high_dim_analysis.plot_mutual_information(
            cluster_data,
            labels,
            output_folder=plot_dir,
            filename=f"mi_{year_range}_k{k}.png"
        )

        from sklearn.feature_selection import f_classif
        X = np.array([cluster_data.inc, cluster_data.raan, cluster_data.sem_maj,
              cluster_data.ecc, cluster_data.mag]).T
        valid = labels != -1  # Exclude noise points
        if np.any(valid) and len(np.unique(labels[valid])) > 1:
            f_vals, _ = f_classif(X[valid], labels[valid])
            mean_f = np.mean(f_vals)
        else:
            mean_f = 0.0  # Avoid invalid or single-cluster cases

        results[-1]["Mean ANOVA F"] = f"{mean_f:.3f}"

    df = pd.DataFrame(results)
    df["Mean ANOVA F"] = pd.to_numeric(df["Mean ANOVA F"], errors="coerce")
    df_sorted = df.sort_values(by="Mean ANOVA F", ascending=False)
    best_params = df_sorted.iloc[0]
    best_k = best_params["k"]

    # Re-run clustering with best parameters to get labels
    best_result, _, _, _, _ = run_clustering(
        k_means, f"K-means (k={k})", normalized_data, data_min, data_max, int(best_k), init='kmeans++'
    )
    best_labels = best_result.labels

    # Plot ANOVA F-values only for best result
    high_dim_analysis.plot_anova_f_values(
        cluster_data,
        best_labels,
        output_folder=plot_dir,
        filename=f"anova_best_{year_range}_k{best_k}.png",
        title=f"ANOVA F-values (Best KMeans Parameters), k = {int(best_k)}"
    )
    print("Best KMeans parameters based on ANOVA F-value:")
    print(best_params)

    display(df)


<>:26: SyntaxWarning: invalid escape sequence '\k'
<>:26: SyntaxWarning: invalid escape sequence '\k'
C:\Users\fionu\AppData\Local\Temp\ipykernel_48812\3664462175.py:26: SyntaxWarning: invalid escape sequence '\k'
  plot_dir = "Images\k_means_running_bins_5d"



Running K-Means for Year Range: 2002-2005
Runtime for k_means: 0.000000 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


Runtime for k_means: 0.000000 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


...

c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


Runtime for k_means: 0.106623 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:42: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest")


Best KMeans parameters based on ANOVA F-value:
Year Range                                            2020-2023
k                                                             5
Runtime (s)                                               0.016
Clusters                                                      5
Points per Cluster    {0: 2035, 1: 543, 2: 605, 3: 447, 4: 742}
Davies-Bouldin                                             None
Calinski-Harabasz                                          None
Dunn Index                                                 None
Silhouette Score                                           None
Mean ANOVA F                                          11898.546
Name: 2, dtype: object


,Year Range,k,Runtime (s),Clusters,Points per Cluster,Davies-Bouldin,Calinski-Harabasz,Dunn Index,Silhouette Score,Mean ANOVA F
0,2020-2023,3,0.004,3,"{0: 1568, 1: 2041, 2: 763}",None,None,None,None,7200.536
1,2020-2023,4,0.016,4,"{0: 2041, 1: 556, 2: 1016, 3: 759}",None,None,None,None,6350.911
2,2020-2023,5,0.016,5,"{0: 2035, 1: 543, 2: 605, 3: 447, 4: 742}",None,None,None,None,11898.546
3,2020-2023,6,0.025,6,"{0: 581, 1: 2033, 2: 743, 3: 428, 4: 66, 5: 521}",None,None,None,None,9461.020
4,2020-2023,7,0.032,7,"{0: 731, 1: 367, 2: 600, 3: 1123, 4: 923, 5: 2...",None,None,None,None,8882.920


# DBSCAN

In [2]:
import cluster_data_5d
from cluster_data_5d import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from cluster_plotter import ClusterPlotter
from clustering_utils_5d import ClusterData
import cluster_plotter
from DBSCAN import dbscan_clustering
import high_dim_analysis

# Define year ranges
standard_year_ranges = {
    f"{start}-{start + 3}": np.arange(start, start + 4)
    for start in [2002, 2006, 2010, 2014, 2019]
}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)  # Include 2023
running_ranges = cluster_data_5d.generate_running_year_ranges(2002, 2023, 4)

binned_data = cluster_data_5d.bin_data_for_clustering(running_ranges, print_res=False)

results_per_year_range = {}

# Output directory for plots
plot_dir = "Images\dbscan_tests_running_bins_5d"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:  
    print(f"\nRunning DBSCAN for Year Range: {year_range}")

    data_array = np.array([cluster_data.inc, cluster_data.raan, cluster_data.sem_maj]).T
    feature_names = ["inclination [°]", "RAAN [°]", "semi major axis [km]"]

    normalized_data, data_min, data_max = normalize_data(data_array)

    # DBSCAN parameter ranges
    eps_values = [0.02, 0.01, 0.015]
    min_samples_values = [10, 15, 25, 30]

    results = []

    for eps in eps_values:
        for min_samples in min_samples_values:
            result_dbscan, time_dbscan, n_clusters_dbscan, points_per_cluster_dbscan, metrics_dbscan = run_clustering(
                dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max, eps=eps, min_samples=min_samples
            )

            unnormalized_data, _ = unnormalize(
                result_dbscan.data, None, data_min, data_max
            )
            # Count noise points
            noise_points = np.sum(result_dbscan.labels == -1)

            results.append({
                "Year Range": year_range,
                "eps": eps,
                "min_samples": min_samples,
                "Runtime (s)": f"{time_dbscan:.3f}",
                "Clusters": n_clusters_dbscan,
                "Points per Cluster": points_per_cluster_dbscan,
                "Noise Points": noise_points,
                "Davies-Bouldin": f"{metrics_dbscan[0]:.3f}" if metrics_dbscan and isinstance(metrics_dbscan[0], (int, float)) else None,
                "Calinski-Harabasz": f"{metrics_dbscan[1]:.3f}" if metrics_dbscan and isinstance(metrics_dbscan[1], (int, float)) else None
            })


            labels = result_dbscan.labels
            """high_dim_analysis.plot_co_membership(
                labels,
                output_folder=plot_dir,
                filename=f"co_membership_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )"""
            high_dim_analysis.plot_correlation_heatmap(
                cluster_data,
                output_folder=plot_dir,
                filename=f"corr_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )
            high_dim_analysis.plot_mutual_information(
                cluster_data,
                labels,
                output_folder=plot_dir,
                filename=f"mi_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )
            
        from sklearn.feature_selection import f_classif
        X = np.array([cluster_data.inc, cluster_data.raan, cluster_data.sem_maj,
              cluster_data.ecc, cluster_data.mag]).T
        valid = labels != -1  # Exclude noise points
        if np.any(valid) and len(np.unique(labels[valid])) > 1:
            f_vals, _ = f_classif(X[valid], labels[valid])
            weights = np.var(X[valid], axis=0)
            weighted_mean_f = np.average(f_vals, weights=weights)
        else:
            weighted_mean_f = 0.0  # Avoid invalid or single-cluster cases

        results[-1]["Weighted Mean ANOVA F"] = f"{weighted_mean_f:.3f}"

    df = pd.DataFrame(results)

    df["Weighted Mean ANOVA F"] = pd.to_numeric(df["Weighted Mean ANOVA F"], errors="coerce")
    df_sorted = df.sort_values(by="Weighted Mean ANOVA F", ascending=False)
    best_params = df_sorted.iloc[0]
    best_eps = best_params["eps"]
    best_min_samples = best_params["min_samples"]

    # Re-run clustering with best parameters to get labels
    best_result, _, _, _, _ = run_clustering(
        dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max,
        eps=best_eps, min_samples=best_min_samples
    )
    best_labels = best_result.labels

    # Plot ANOVA F-values only for best result
    high_dim_analysis.plot_anova_f_values(
        cluster_data,
        best_labels,
        output_folder=plot_dir,
        filename=f"anova_best_{year_range}_eps{best_eps}_minsamples_{best_min_samples}.png",
        title=f"ANOVA F-values (Best DBSCAN Parameters), eps = {float(best_eps)}, minsamples = {float(best_min_samples)}"
    )

    print("Best DBSCAN parameters based on ANOVA F-value:")
    print(best_params)
    display(df)

<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:26: SyntaxWarning: invalid escape sequence '\d'
C:\Users\fionu\AppData\Local\Temp\ipykernel_48812\3786361329.py:26: SyntaxWarning: invalid escape sequence '\d'
  plot_dir = "Images\dbscan_tests_running_bins_5d"



Running DBSCAN for Year Range: 2002-2005
Runtime for dbscan_clustering: 0.283700 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Das System kann die angegebene Datei nicht finden
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\fionu\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\fionu\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\fionu\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\fionu\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.C

Runtime for dbscan_clustering: 0.222295 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


...

c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


Runtime for dbscan_clustering: 0.031616 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:42: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest")


Best DBSCAN parameters based on ANOVA F-value:
Year Range                                                       2020-2023
eps                                                                   0.01
min_samples                                                             30
Runtime (s)                                                          0.030
Clusters                                                                12
Points per Cluster       {0: 417, 1: 89, 2: 70, 3: 234, 4: 41, 5: 165, ...
Noise Points                                                          2702
Davies-Bouldin                                                        None
Calinski-Harabasz                                                     None
Weighted Mean ANOVA F                                            54284.893
Name: 7, dtype: object


,Year Range,eps,min_samples,Runtime (s),Clusters,Points per Cluster,Noise Points,Davies-Bouldin,Calinski-Harabasz,Weighted Mean ANOVA F
0,2020-2023,0.020,10,0.045,28,"{0: 52, 1: 484, 2: 1797, 3: 17, 4: 14, 5: 151,...",1272,None,None,NaN
1,2020-2023,0.020,15,0.051,14,"{0: 50, 1: 484, 2: 1771, 3: 151, 4: 217, 5: 26...",1483,None,None,NaN
2,2020-2023,0.020,25,0.032,8,"{0: 484, 1: 1592, 2: 143, 3: 123, 4: 211, 5: 6...",1732,None,None,NaN
3,2020-2023,0.020,30,0.044,7,"{0: 481, 1: 1573, 2: 142, 3: 118, 4: 211, 5: 6...",1786,None,None,16685.002
4,2020-2023,0.010,10,0.024,22,"{0: 10, 1: 446, 2: 231, 3: 11, 4: 37, 5: 310, ...",2060,None,None,NaN
5,2020-2023,0.010,15,0.022,17,"{0: 424, 1: 183, 2: 24, 3: 25, 4: 302, 5: 94, ...",2223,None,None,NaN
6,2020-2023,0.010,25,0.014,13,"{0: 419, 1: 173, 2: 281, 3: 43, 4: 33, 5: 230,...",2537,None,None,NaN
7,2020-2023,0.010,30,0.030,12,"{0: 417, 1: 89, 2: 70, 3: 234, 4: 41, 5: 165, ...",2702,None,None,54284.893
8,2020-2023,0.015,10,0.039,23,"{0: 10, 1: 480, 2: 647, 3: 11, 4: 881, 5: 17, ...",1621,None,None,NaN
9,2020-2023,0.015,15,0.030,12,"{0: 477, 1: 632, 2: 420, 3: 17, 4: 421, 5: 137...",1862,None,None,NaN
